In [1]:
import os
from elasticsearch import Elasticsearch 
import random
from collections import OrderedDict
from nltk.stem import PorterStemmer
import numpy
from sklearn.metrics import accuracy_score

In [11]:
import elasticsearch_dsl

In [2]:
es=Elasticsearch()

In [3]:
#Reading the file with mapping for each file with respect to being spam or ham
spam_dict={}
index_path='../full/trec07p/full/'
for line in os.listdir(index_path):
    file=open(index_path+line,"r", encoding="ISO-8859-1")
    for line in file:
        line=line.strip().split(' ')
        fname=line[1].split('/')
        spam_dict[fname[2]]=line[0]

In [4]:
#func to get a list of spam and ham files
def get_spam_ham():
    spams=[]
    hams=[]
    for file in spam_dict.keys():
        if spam_dict[file]=='spam':
            spams.append(file)
        else:
            hams.append(file)
    return spams,hams
        
    

In [5]:
index_name='trec_spam'

In [7]:
#creating of list of possible spam terms
features=list(set(['buy','miracle','virus','refinance','performance','prize','credit','million',"hot","naked", "fast","viagra","job", "lose","win","million","limited", "sex",
 'diagnostics','profit','shopper','insurance','only','incredible','celebrity','meet','act','dollars','price','double','medication','here','chance','cialis','viagra','join','click','unsolicited','order',
 'singles', '100','spam','percent','rate','ad','luxury','subscribe','freedom','bonus','free','fast','limited','hidden','now','earn','dollar',
 'visit','medicine','weight','money','affordable','legal','trial','deal','valium','loans','lose','win','malware','discount','interest','instant','collect','no',
 'cash','remove','claim','website','compare','check','success','love','sales','home','clearance','sex','c','lower','our','extra',
"enlargement","100","penis","singles","luxury","erectile","impotence","erection","hurry","Cheapest"]))
len(features)

90

In [12]:
#creating doc mapping and inverse doc mapping to ease of reading from file
s = elasticsearch_dsl.Search(using=es, index=index_name, doc_type='document')
#print(s)
s = s.source([])  # only get ids, otherwise `fields` takes a list of field names
docs = [h.meta.id for h in s.scan()]
doc_map={doc:i+1 for i,doc in enumerate(docs)}
inv_doc_map={i+1:doc for i,doc in enumerate(docs)}


In [25]:
#opening file to store training and test set
train_file=open('../Files/train_set.txt','a')
test_file=open('../Files/test_set.txt','a')

In [53]:
#creating spam dictionary with each feature as key and ids as value
def update_spamdict(dataset,spam_words):
    spamdict={}
    for word in spam_words:
        body = {
          "query": { 
            "bool": { 
               "must": [
                   { "match": {"split":dataset}}, 
                       ],
            "filter": [ 
                 { "term":  {"body":word }}
              ]
           }
       },
            "size": 75419
        }
        res = es.search(index=index_name, body=body)
        ids = [d['_id'] for d in res['hits']['hits']]
        spamdict[word]=ids
    return spamdict
#created the training and testing matrix
def create_matrix(dataset,doc_to_id,spam_words):
  #  spamdict=update_spamdict(dataset,spam_words)
    if dataset=='training'  :
        spamdict=update_spamdict('training',spam_words)
        ftrain=open("../Files/train_set.txt","w")
        body={
          "query": {
              "term" : { "split" : "training" } 
                   },
                  "size": 75419}
        res = es.search(index=index_name, body=body)
        ids = [d['_id'] for d in res['hits']['hits']]
        count=0
        print("Started writing into file for training")
        for each_id in ids:
            
            temp_list=[]
            for each_word in spam_words:
                try:
                    if each_id in spamdict[each_word]:
                         temp_list.append("1")
                    else: 
                         temp_list.append("0")
                except:
                    temp_list.append("0")
            label=es.get(index=index_name, doc_type='document', id=each_id)['_source']['label']
            if label=='yes':
                spameval=1
            else:
                spameval=0
            binary_list=" ".join(temp_list)
            ftrain.write(str(doc_to_id[each_id]) + " "+str(spameval)+" "+binary_list+"\n")
            count+=1
            if (count%1000==0):
                print("printed:"+str(count))
        ftrain.close()
        print("completed for training")
    else:
        spamdict=update_spamdict('testing',spam_words)
        ftest=open("../Files/test_set.txt","w")
        body={
          "query": {
              "term" : { "split" : "testing" } 
                   },
                  "size": 75419}
        res = es.search(index=index_name, body=body)
        ids = [d['_id'] for d in res['hits']['hits']]
        count=0
        print("started writing for testing")
        for each_id in ids:
            
            temp_list=[]
            for each_word in spam_words:
                try:
                    if each_id in spamdict[each_word]:
                        temp_list.append("1")
                    else:
                        temp_list.append("0")
                except:
                        temp_list.append("0")
            label=es.get(index=index_name, doc_type='document', id=each_id)['_source']['label']
            if label=='yes':
                spameval=1
            else:
                spameval=0
            binary_list=" ".join(temp_list)
            ftest.write(str(doc_to_id[each_id]) + " "+str(spameval)+" "+binary_list+"\n")
            count+=1
            if count%1000==0:
                print("printed:"+str(count))
        ftest.close()
        print("completed for test")


In [54]:
create_matrix('training',doc_map,features)
create_matrix('test',doc_map,features)

Started writing into file for training
printed:1000
printed:2000
printed:3000
printed:4000
printed:5000
printed:6000
printed:7000
printed:8000
printed:9000
printed:10000
printed:11000
printed:12000
printed:13000
printed:14000
printed:15000
printed:16000
printed:17000
printed:18000
printed:19000
printed:20000
printed:21000
printed:22000
printed:23000
printed:24000
printed:25000
printed:26000
printed:27000
printed:28000
printed:29000
printed:30000
printed:31000
printed:32000
printed:33000
printed:34000
printed:35000
printed:36000
printed:37000
printed:38000
printed:39000
printed:40000
printed:41000
printed:42000
printed:43000
printed:44000
printed:45000
printed:46000
printed:47000
printed:48000
printed:49000
printed:50000
printed:51000
printed:52000
printed:53000
printed:54000
printed:55000
printed:56000
printed:57000
printed:58000
printed:59000
printed:60000
completed for training
started writing for testing
printed:1000
printed:2000
printed:3000
printed:4000
printed:5000
printed:6000
p

In [55]:
train_set_file=numpy.loadtxt('C:/Users/Nikhar/Downloads/Side_work/spam/Files/train_set.txt', usecols=range(2,92))
train_set_label=numpy.loadtxt('C:/Users/Nikhar/Downloads/Side_work/spam/Files/train_set.txt', usecols=(1))
test_set_file=numpy.loadtxt('C:/Users/Nikhar/Downloads/Side_work/spam/Files/test_set.txt', usecols=range(2,92))

In [56]:
test_set_label=numpy.loadtxt('C:/Users/Nikhar/Downloads/Side_work/spam/Files/test_set.txt', usecols=1)

In [61]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
test_result = gnb.fit(train_set_file,train_set_label).predict(test_set_file)

In [62]:
#accuracy using gaussian naive bayes
accuracy_score(test_result,test_set_label)

0.5574118271015646

In [63]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
test_result = clf.fit(train_set_file,train_set_label).predict(test_set_file)

In [64]:
#accuracy using decision tree
accuracy_score(test_result,test_set_label)

0.9669185892336251